In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Import dataset

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dataset = pd.read_csv('/kaggle/input/titanic/train.csv')
dataset.shape

(891, 12)

In [4]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
dataset.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
dataset['Cabin'].isnull().sum() / 891

0.7710437710437711

`Cabin` column is approx 77% empty. Let's remove from the training dataset

## Data Preprocessing

In [8]:
import re

def extract_title(name: str):
    title_search = re.search(' ([A-Za-z]+)\.', name)

    if title_search:
        return title_search.group(1)
    else:
        return ""

In [9]:
dataset['Title'] = dataset['Name'].apply(extract_title)
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [10]:
cat_features = [col for col in dataset.columns if dataset[col].dtype=='object' and col!='Name']
num_features = [col for col in dataset.columns if dataset[col].dtype!='object' and col!='PassengerId']
cat_features.remove('Cabin')
num_features.remove('Survived')

num_features, cat_features

(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
 ['Sex', 'Ticket', 'Embarked', 'Title'])

In [11]:
X = dataset[num_features + cat_features]
y = dataset['Survived']

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


cat_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
    ]
)
num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

## Data Splitting

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                 test_size=0.25,
                                                 stratify=y,
                                                 random_state=7 
                                                 )

## Model Training

### Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

lr_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])
lr_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier', LogisticRegression())])

In [15]:
y_pred = lr_model.predict(X_val)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [16]:
# accuracy score
from sklearn.metrics import accuracy_score

accuracy_score(y_val, y_pred)

0.8385650224215246

In [17]:
# Making confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val, y_pred)
cm

array([[122,  15],
       [ 21,  65]])

### Decision Tree Classification

In [18]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
dt_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])
dt_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier',
                 DecisionTreeClassifier(criterion='entropy', random_state=0))])

In [19]:
y_pred = dt_model.predict(X_val)

acc_score = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
acc_score, cm

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(0.8251121076233184,
 array([[118,  19],
        [ 20,  66]]))

### RandomForest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators = 200,
                                    criterion='entropy',
                                    random_state = 0)
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', rf_classifier)
])
rf_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', n_estimators=200,
                                        random_state=0))])

In [21]:
y_pred = rf_model.predict(X_val)

acc_score = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
acc_score, cm

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(0.852017937219731,
 array([[125,  12],
        [ 21,  65]]))

### RandomForest HyperParameter Tuning

In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(estimator=rf_model,
                           param_grid=param_grid, 
                           cv=5, n_jobs=-1,
                           scoring='accuracy',
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        handle_unknown='ignore'))]),
                                                                         ['Sex',
                                                                          'Ticket',
                                                                          'Embarked',
                                                                          'Title'])])),
                                       ('classifier',
                                        RandomForestClassifier(criterion='entropy',
                                                               n_estimators=200,
                                                               random_state=0))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=1)

In [23]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

({'classifier__max_depth': None,
  'classifier__min_samples_split': 5,
  'classifier__n_estimators': 300},
 0.8487262933453035)

### SVM Classification

In [24]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf', gamma='scale')
svc_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', svc)
])
svc_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier', SVC())])

In [25]:
y_pred = svc_model.predict(X_val)

acc_score = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
acc_score, cm

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(0.8340807174887892,
 array([[122,  15],
        [ 22,  64]]))

### SVM Classification Hyperparameter Tuning

In [26]:
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__gamma': [1, 0.1, 0.01, 0.001],
    'classifier__kernel': ['rbf', 'poly', 'sigmoid']
}

grid_search = GridSearchCV(estimator=svc_model,
                           param_grid=param_grid, 
                           cv=5, n_jobs=-1,
                           scoring='accuracy',
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        handle_unknown='ignore'))]),
                                                                         ['Sex',
                                                                          'Ticket',
                                                                          'Embarked',
                                                                          'Title'])])),
                                       ('classifier', SVC())]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.1, 1, 10, 100],
                         'classifier__gamma': [1, 0.1, 0.01, 0.001],
                         'classifier__kernel': ['rbf', 'poly', 'sigmoid']},
             scoring='accuracy', verbose=1)

In [27]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

({'classifier__C': 100,
  'classifier__gamma': 0.01,
  'classifier__kernel': 'rbf'},
 0.850241274828863)

### KNN Classification

In [28]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(weights='distance')
knn_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', knn)
])
knn_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier', KNeighborsClassifier(weights='distance'))])

In [29]:
y_pred = knn_model.predict(X_val)

acc_score = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
acc_score, cm

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(0.7937219730941704,
 array([[114,  23],
        [ 23,  63]]))

### KNN Classification HyperParameter Tuning

In [30]:
param_grid = {
    'classifier__n_neighbors': [3, 5, 7, 9],
    'classifier__weights': ['uniform', 'distance'],
    # 'classifier__p': [1,2,3],
}

grid_search = GridSearchCV(estimator=knn_model,
                           param_grid=param_grid, 
                           cv=5, n_jobs=-1,
                           scoring='accuracy',
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        handle_unknown='ignore'))]),
                                                                         ['Sex',
                                                                          'Ticket',
                                                                          'Embarked',
                                                                          'Title'])])),
                                       ('classifier',
                                        KNeighborsClassifier(weights='distance'))]),
             n_jobs=-1,
             param_grid={'classifier__n_neighbors': [3, 5, 7, 9],
                         'classifier__weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [31]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score

({'classifier__n_neighbors': 9, 'classifier__weights': 'distance'},
 0.8367186623274605)

## Model Selection

Out of all the models `RandomForest` and `SVM` seems to be having more accuracy.
Lets compare both of the best models.

In [32]:
rf_classifier = RandomForestClassifier(n_estimators = 300,
                                    min_samples_split=5,
                                    criterion='entropy',
                                    random_state = 0)
rf_best_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', rf_classifier)
])
rf_best_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier',
                 RandomForestClassifier(criterion='entropy',
                                        min_samples_split=5, n_estimators=300,
                                        random_state=0))])

In [33]:
svc = SVC(kernel='rbf', gamma=0.01, C=100)
svc_best_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', svc)
])
svc_best_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Ticket', 'Embarked',
                                                   'Title'])])),
                ('classifier', SVC(C=100, gamma=0.01))])

In [34]:
y_pred = rf_best_model.predict(X_val)

acc_score = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
acc_score, cm

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(0.8340807174887892,
 array([[125,  12],
        [ 25,  61]]))

In [35]:
y_pred = svc_best_model.predict(X_val)

acc_score = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
acc_score, cm

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(0.852017937219731,
 array([[124,  13],
        [ 20,  66]]))

`SVM Classification` model has more accuracy and less `false-negative` and `false-positive`. So, it will most likely to perform better.

In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.91      0.88       137
           1       0.84      0.77      0.80        86

    accuracy                           0.85       223
   macro avg       0.85      0.84      0.84       223
weighted avg       0.85      0.85      0.85       223



## Score the Test Dataset

In [37]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [38]:
test_df.shape

(418, 11)

In [39]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [40]:
test_df['Title'] = test_df['Name'].apply(extract_title)
X_test = test_df[num_features+cat_features]
predictions = rf_model.predict(X_test)
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'],
                          'Survived': predictions})
submission.tail()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,1


In [41]:
submission.shape

(418, 2)

In [42]:
submission.to_csv('submission.csv', index=False)